In [45]:
import oci
import uuid
import base64
import json
import ocifs
import pandas as pd
import numpy as np

config = oci.config.from_file('config_uk')
fs = ocifs.OCIFileSystem()

# Compartment where processor job will be created
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaal63rmctooj..."

COMPROVANTE_MODEL_ID = "ocid1.aidocumentmodel.oc1.uk-london-1.amaaaaaat..."
Doc_MODEL_ID = "ocid1.aidocumentmodel.oc1.uk-london-1.amaaaaaats..."
Doc_RECORTADO_MODEL_ID = "ocid1.aidocumentmodel.oc1.uk-london-1.amaaaaaatsbrckqapwz..."

# Service endpoint
endpoint = "https://document-preprod.aiservice.uk-london-1.oci.oraclecloud.com"

def create_processor_job_callback(times_called, response):
    #print("Waiting for processor lifecycle state to go into succeeded state:", response.data)
    pass

In [2]:
print(oci.__version__)

2.88.2+preview.1.5970


In [3]:
document_list = fs.glob("oci://xablau@xxxxxxxxxx/*.jpg")

In [54]:
def Analisa_Comprovante(comprovante_path):
    
    with fs.open(comprovante_path) as f:
        content = f.read()

        encoded_string = base64.b64encode(content, altchars=None)
        key_value_detection_sample_string = encoded_string.decode("utf-8", "ignore")
    
    aiservicedocument_client = oci.ai_document.AIServiceDocumentClientCompositeOperations(oci.ai_document.AIServiceDocumentClient(config=config))

    # Document Key-Value Detection Feature
    key_value_detection_feature = oci.ai_document.models.DocumentKeyValueExtractionFeature(model_id="ocid1.aidocumentmodel.oc1.uk-london-1.amaaaaaatsbrckqav75tvil233tzcopjz6owieufqbpceggbfwsmjnopczra")

    # Setup the output location where processor job results will be created
    output_location = oci.ai_document.models.OutputLocation()
    output_location.namespace_name = "id3kyspkytmr"  
    output_location.bucket_name = "Comprovante_Output"  
    output_location.prefix = "Output_Comprovante_BancoPan" 

    # Create a processor_job for key_value_detection feature
    create_processor_job_details_key_value_detection = oci.ai_document.models.CreateProcessorJobDetails(
                                                        display_name=str(uuid.uuid4()),
                                                        compartment_id=COMPARTMENT_ID,
                                                        input_location=oci.ai_document.models.InlineDocumentContent(data=key_value_detection_sample_string),
                                                        output_location=output_location,
                                                        processor_config=oci.ai_document.models.GeneralProcessorConfig(features=[key_value_detection_feature]))

    create_processor_response = aiservicedocument_client.create_processor_job_and_wait_for_state(
        create_processor_job_details=create_processor_job_details_key_value_detection,
        wait_for_states=[oci.ai_document.models.ProcessorJob.LIFECYCLE_STATE_SUCCEEDED],
        waiter_kwargs={"wait_callback": create_processor_job_callback})
 
    processor_job: oci.ai_document.models.ProcessorJob = create_processor_response.data

    print("Getting defaultObject.json from the output_location")
    object_storage_client = oci.object_storage.ObjectStorageClient(config=config)
    get_object_response = object_storage_client.get_object(namespace_name=output_location.namespace_name,
                                                           bucket_name=output_location.bucket_name,
                                                           object_name="{}/{}/_/results/defaultObject.json".format(
                                                               output_location.prefix, processor_job.id))
    response = get_object_response.data.content.decode()
    
    json_response = json.loads(response)
    
    return json_response

In [55]:
def Analisa_Doc(Doc_path):
    
    with fs.open(Doc_path) as f:
        content = f.read()

        encoded_string = base64.b64encode(content, altchars=None)
        key_value_detection_sample_string = encoded_string.decode("utf-8", "ignore")

    aiservicedocument_client = oci.ai_document.AIServiceDocumentClientCompositeOperations(oci.ai_document.AIServiceDocumentClient(config=config))

    # Document Key-Value Detection Feature
    key_value_detection_feature = oci.ai_document.models.DocumentKeyValueExtractionFeature(model_id = "ocid1.aidocumentmodel.oc1.uk-london-1.amaaaaaatsbrckqa56v5wx5qlsta2qnjgfqcxofpxbvmebcg6wszu6rszhuq")

    # Setup the output location where processor job results will be created
    output_location = oci.ai_document.models.OutputLocation()
    output_location.namespace_name = "id3kyspkytmr"  
    output_location.bucket_name = "Doc_Output"  
    output_location.prefix = "Output_Doc_BancoPan" 

    # Create a processor_job for key_value_detection feature
    create_processor_job_details_key_value_detection = oci.ai_document.models.CreateProcessorJobDetails(
                                                        display_name=str(uuid.uuid4()),
                                                        compartment_id=COMPARTMENT_ID,
                                                        input_location=oci.ai_document.models.InlineDocumentContent(data=key_value_detection_sample_string),
                                                        output_location=output_location,
                                                        processor_config=oci.ai_document.models.GeneralProcessorConfig(features=[key_value_detection_feature]))
    
    create_processor_response = aiservicedocument_client.create_processor_job_and_wait_for_state(
        create_processor_job_details=create_processor_job_details_key_value_detection,
        wait_for_states=[oci.ai_document.models.ProcessorJob.LIFECYCLE_STATE_SUCCEEDED],
        waiter_kwargs={"wait_callback": create_processor_job_callback})
 
    processor_job: oci.ai_document.models.ProcessorJob = create_processor_response.data

    print("Getting defaultObject.json from the output_location")
    object_storage_client = oci.object_storage.ObjectStorageClient(config=config)
    get_object_response = object_storage_client.get_object(namespace_name=output_location.namespace_name,
                                                           bucket_name=output_location.bucket_name,
                                                           object_name="{}/{}/_/results/defaultObject.json".format(
                                                               output_location.prefix, processor_job.id))
    response = get_object_response.data.content.decode()
    json_response = json.loads(response)
    
    return json_response


In [ ]:
path_comprovante = comprovante_list[0:10]
for path in path_comprovante:
    response_comprovante = Analisa_Comprovante(path)
    for i in range(len(response_comprovante['pages'][0]['documentFields'])):
        entidade = response_comprovante['pages'][0]['documentFields'][i]['fieldLabel']['name']
        valor = response_comprovante['pages'][0]['documentFields'][i]['fieldValue']['value']

        print(f"{entidade}: {valor}")